In [6]:
import os
from dotenv import load_dotenv, find_dotenv
from llama_index import VectorStoreIndex, SimpleDirectoryReader
_ = load_dotenv(find_dotenv())

In [7]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)

Multiple definitions in dictionary at byte 0xf93db for key /MediaBox
Multiple definitions in dictionary at byte 0xdbf76 for key /MediaBox
Multiple definitions in dictionary at byte 0xeaa9 for key /MediaBox
Multiple definitions in dictionary at byte 0x21a1b for key /GSP1
Multiple definitions in dictionary at byte 0x21bc8 for key /GSP1
Multiple definitions in dictionary at byte 0x21db1 for key /GSP1
Multiple definitions in dictionary at byte 0x21f9a for key /GSP1
Multiple definitions in dictionary at byte 0x221d8 for key /GSP1
Multiple definitions in dictionary at byte 0x223c5 for key /GSP1
Multiple definitions in dictionary at byte 0x2256c for key /GSP1
Multiple definitions in dictionary at byte 0x2272c for key /GSP1


In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("deepspeek是什么？")
print(response)

I'm sorry, but I cannot provide an answer to your query as there is no information given in the context about "deepspeek".


In [12]:
index.storage_context.persist()

In [17]:
from llama_index.indices.postprocessor import LLMRerank
from llama_index.retrievers import VectorIndexRetriever, BaseRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import ServiceContext, get_response_synthesizer
from llama_index.indices.query.schema import QueryBundle

# 创建自定义检索器
class LLMRerankRetriever(BaseRetriever):
    def __init__(self, vector_retriever: VectorIndexRetriever, service_context: ServiceContext):
        self._vector_retriever = vector_retriever
        self._service_context = service_context
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle):
        retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
        reranker = LLMRerank(service_context=self._service_context)
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
        return retrieved_nodes

# 创建自定义检索器实例
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3)
custom_retriever = LLMRerankRetriever(vector_retriever, ServiceContext.from_defaults())

# 创建自定义查询引擎
response_synthesizer = get_response_synthesizer()
custom_query_engine = RetrieverQueryEngine(retriever=custom_retriever, response_synthesizer=response_synthesizer)

# 查询
query = "transformers是什么？"
results = custom_query_engine.query(query)
print(results.response)

transformers是一个Python库，用于自然语言处理（NLP）中的深度学习模型。它提供了各种预训练模型，包括文本分类、命名实体识别、问答系统等任务。该库还提供了用于加载、保存和使用这些预训练模型的工具和函数。
